In [1]:
from dataclasses import dataclass
from enum import Enum, IntEnum
from typing import List, Optional
import random
import time
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from perlin_noise import PerlinNoise


In [26]:
BLACK = "\033[90m"
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
MAGENTA = "\033[95m"
CYAN = "\033[96m"
RESET = "\033[0m"

class Inventory(IntEnum):
    LEAF = 1
  
@dataclass
class Position:
    x: int
    y: int
    MAX_X = 19
    MAX_Y = 19
    
    def random(max_x: int = MAX_X, max_y: int = MAX_Y) -> 'Position':
        x = random.randrange(max_x)
        y = random.randrange(max_y)
        return Position(x, y)
    
    def __add__(self, other: 'Position') -> 'Position':
        return Position(self.x + other.x, self.y + other.y)
    
    def __sub__(self, other: 'Position') -> 'Position':
        return Position(self.x - other.x, self.y - other.y)

@dataclass
class Direction:
    UP = Position(0,1)
    UP_RIGHT = Position(1,1)
    RIGHT = Position(1,0)
    DOWN_RIGHT = Position(1,-1)
    DOWN = Position(0,-1)
    DOWN_LEFT = Position(-1,-1)
    LEFT = Position(-1,0)
    UP_LEFT = Position(-1,1)
    STAY_STILL = Position(0,0)
    
    @staticmethod
    def random() -> 'Position':
        directions = [Direction.STAY_STILL, Direction.UP, Direction.UP_RIGHT, Direction.RIGHT, Direction.DOWN_RIGHT,
                      Direction.DOWN, Direction.DOWN_LEFT, Direction.LEFT, Direction.UP_LEFT]
        return random.choice(directions)
  

@dataclass
class Size:
    width: int
    height: int

class Caste(IntEnum):
    WORKER = 1
    SOLDIER = 2
    QUEEN = 3

    def __str__(self):
        match self:
            case Caste.WORKER:
                return "W"
            case Caste.SOLDIER:
                return "S"
            case Caste.QUEEN:
                return "Q"
            
    @classmethod
    def random(cls) -> 'Caste':
        int_value = random.randrange(len(Caste))
        return scls(int_value)

@dataclass
class Ant():
    caste: Caste
    position: Position
    age: int
    health: int
    food: int
    water: int
    inventory: List[Inventory]
    history: List[str]
    
    MAX_X = 19
    MAX_Y = 19
    
    def move(self, direction: Direction):
        self.position.x = (self.position.x + direction.x) % self.MAX_X
        self.position.y = (self.position.y + direction.y) % self.MAX_Y
    
    def system_prompt(self) -> str:
        return """
    You are an ant. You can move around and collect food and water.
    You can move UP, UP_RIGHT, RIGHT, DOWN_RIGHT, DOWN, DOWN_LEFT, LEFT, LEFT_UP, or STAY_STILL.
    You can collect food and water by moving onto them.
    Move to the food and take it back to the nest.
    
    """
        
    def next_prompt(self) -> str:
        return ""


class TerraineItem(Enum):
    GRASS = 0
    DIRT = 1
    WATER = 2
    TREE = 3
    NEST = 4
    FOOD = 5

    def color(self) -> str:
        match self:
            case TerraineItem.GRASS:
                return '\x1b[48;2;0;255;0m'
            case TerraineItem.DIRT:
                return '\x1b[48;2;255;64;64m'
            case TerraineItem.WATER:
                return '\x1b[48;2;0;0;255m'
            case TerraineItem.TREE:
                return '\x1b[48;2;255;64;64m'
            case TerraineItem.NEST:
                return '\x1b[48;2;128;64;64m'
            case TerraineItem.FOOD:
                return '\x1b[48;2;10;10;10m'
            

def generate_terraine(size: Size) -> List[List[TerraineItem]]:
    def mapping(v: float) -> TerraineItem:
        if not isinstance(v, float):
            assert False, f"v is not a float: {v}"
        if v < -0.25:
            return TerraineItem.WATER
        else:
            if random.random() < 0.01:
                return TerraineItem.FOOD
            else:
                return TerraineItem.GRASS
   
    noise = PerlinNoise(octaves=5, seed=1)
    xpix, ypix = size.width, size.height
    noise_map = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]

    terraine = [[mapping(v) for v in row] for row in noise_map]
    for i in range(2):
        for j in range(2):
            terraine[i-1+ypix//2][j-1+xpix//2] = TerraineItem.NEST
            
    return terraine
    
@dataclass
class Map():
    size: Size
    terraine: List[TerraineItem]
    
    def __post_init__(self):
        self.terraine = generate_terraine(self.size)

    def set_item(self, pos:Position, item:TerraineItem):
        index = self.size.width * pos.y + pos.x
        self.terraine[index] = item

    def get_item(self, pos: Position) -> TerraineItem:
        index = self.size.width * pos.y + pos.x
        return self.terraine[index]


@dataclass
class World():
    ants: List[Ant]
    map: Map
    
    def ant_at(self, position: Position) -> Optional[Ant]:
        for ant in self.ants:
            if ant.position == position:
                return ant
        return None

    def display(self) -> str:
        display = ""
        for y, row in enumerate(self.map.terraine):
            for x, item in enumerate(row):
                display += item.color()
                if self.ant_at(Position(x,y)):
                    display += f"{BLACK}{ant.caste}{RESET}"
                else:
                    display += "\n"
        return display
    
    @staticmethod
    def create(map_size: Size = Size(20, 20)) -> 'World':
    
        ants = []
        for _ in range(5):
            pos = Position.random(map_size.width, map_size.height)
            ant = Ant(Caste.WORKER, pos, 0, 100, 100, 100, [], [])
            ants.append(ant)

        terraine = [TerraineItem.GRASS for _ in range(map_size.width*map_size.height)]
        map = Map(map_size, terraine)

        return World(ants, map)
    

In [29]:
world = World.create()

for _ in range(100):
    for ant in world.ants:
        direction = Direction.random()
        ant.move(direction)
    print(world.display())
    clear_output(wait=True)
    time.sleep(1)
    

KeyboardInterrupt: 